In [2]:
%pip install opencv-python
%pip install matplotlib
%pip install python-dotenv
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/999.9 kB ? eta -:--:--
   ---------------------------------------- 999.9/999.9 kB 23.4 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 37.5 MB/s  0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ----------------------------------- ---- 7/8 [openai]
   ----------------------------------- ---- 7/8 [openai]
   ----------------------------------- ---- 7/8 [openai]
   ----------------------------------- ---- 7/8 [openai]
   ----------------------------------- ---- 7/8 [ope

In [8]:
# imports
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
import openai

load_dotenv()

True

In [4]:
def split_comic_panels(comic_path, output_path):
    image = cv2.imread(Path(comic_path))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    bounding_rects = []
    panels = []
    for contour in contours:
        if cv2.contourArea(contour) > 1000:
            x, y, w, h = cv2.boundingRect(contour)
            bounding_rects.append((x, y, w, h))
    
    bounding_rects.sort(key=lambda x: x[0]//10)
    bounding_rects.sort(key=lambda x: x[1]//10)
    
    for (x, y, w, h) in bounding_rects:
        panel = image[y:y+h, x:x+w]
        panels.append(panel)
        cv2.imwrite(Path(f"{output_path}/{len(panels)-1}.png"), panel)


split_comic_panels("smbc-beautiful-4.png", "./temp-comic-panels/")

In [44]:
client = openai.OpenAI()
# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

# Getting the file ID
file_id = create_file(Path("./smbc-much.png"))


prompt =  """
Please transcribe this comic in the same form as the following example transcription:
### Panel 1
Red-haired woman: No! No more saying irregardless to mean regardless! You're double-negating with the irr and the less.
### Panel 2
Grey-haired woman: You misheard. I said irrirregardless. Double the irr. So it's back to meaning regardless.
### Panel 3
Grey-haired woman: As long as I employ an even number of irrs, my word remains lexically valid.
### Panel 4
Red-haired woman: But...why?
Grey-haired woman: I use it as an alarm noise when people are too prescriptivist about language.
### Panel 5
*silhouetted*
Red-haired woman: That's still not proper-
Grey-haired woman: Irrirrirrirrirr
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": prompt},
            {
                "type": "input_image",
                "file_id": file_id,
            },
        ],
    }],
)


print(response.output_text)


### Panel 1
Red-haired woman: I think women like sex as much as men.
### Panel 2
Brown-haired man: Hm.
### Panel 3
Brown-haired man: How would you react if I stopped contributing to housework, childcare, and bills for six months, and after stewing the entire time, you decided to finally confront me, at which point I licked my lips, looked you in the eyes and said "Sounds like I need a spanking" because I CAN TELL YOU MY REACTION WOULD REGRETTABLY BE VERY POSITIVE.
### Panel 4
Red-haired woman: Well, if we stipulate that self-respect needs to not be disastrously annihila-
### Panel 5
*silhouetted*
Brown-haired man: Don't you narrow my horizons!
